In [16]:
import re
from itertools import zip_longest

import numpy as np
from aocd.models import Puzzle
import pickle
import pyperclip

from collections import deque

In [2]:
year, day = 2024, 10

In [3]:
puzzle = Puzzle(year=year, day=day)

In [4]:
puzzle.examples[0].answer_a

'36'

In [5]:
example = puzzle.examples[0].input_data
print(example)

0123
1234
8765
9876


In [74]:
def solution_a(data: str) -> tuple[str, str, int]:
    # a good hiking trail is as long as possible and has an even, gradual, uphill slope.
    # For all practical purposes, this means that a hiking trail is any path that starts at height 0, 
    # ends at height 9, and always increases by a height of exactly 1 at each step. 
    # Hiking trails never include diagonal steps - only up, down, left, or right (from the perspective of the map).
    # A trailhead is any position that starts one or more hiking trails - here, 
    # these positions will always have height 0. 
    # A trailhead's score is the number of 9-height positions reachable from that trailhead via a hiking trail.
    hiking_map = np.array([list(map(int, list(line))) for line in data.splitlines()])
    n, c = hiking_map.shape
    print(hiking_map)
    score = 0
    
    trailheads = np.argwhere(hiking_map == 0)

    def reachable_peaks(start_pos):
        peaks = set()
        points_to_visit = deque([start_pos])
    
        while points_to_visit:
            current_pos = points_to_visit.pop()
            height = hiking_map[tuple(current_pos)]
    
            for x,y in [(-1,0), (1,0), (0,-1), (0,1)]:
                new_pos = current_pos + np.array([x,y])
    
                if 0 <= new_pos[0] < n and 0 <= new_pos[1] < c:
                    if hiking_map[tuple(new_pos)] == 9 and height==8:
                        peaks.add(tuple(new_pos))
                    elif hiking_map[tuple(new_pos)] == height + 1:
                        points_to_visit.append(new_pos)

        return peaks

    peaks_per_trailhead = [(trailhead, reachable_peaks(trailhead)) for trailhead in trailheads]

    # print out reachable peaks
    hiking_map = hiking_map.astype(str)
    for trailhead, peaks in peaks_per_trailhead:
        for peak in peaks:
            hiking_map[tuple(peak)] = "X"
    print(hiking_map)

    return sum(len(x[1]) for x in peaks_per_trailhead)

In [75]:
solution_a(puzzle.examples[0].input_data)

[[0 1 2 3]
 [1 2 3 4]
 [8 7 6 5]
 [9 8 7 6]]
[['0' '1' '2' '3']
 ['1' '2' '3' '4']
 ['8' '7' '6' '5']
 ['X' '8' '7' '6']]


1

In [76]:
assert solution_a(puzzle.examples[0].input_data) == 1

[[0 1 2 3]
 [1 2 3 4]
 [8 7 6 5]
 [9 8 7 6]]
[['0' '1' '2' '3']
 ['1' '2' '3' '4']
 ['8' '7' '6' '5']
 ['X' '8' '7' '6']]


In [77]:
input_ = """...0...
...1...
...2...
6543456
7.....7
8.....8
9.....9""".replace(".","3")
assert solution_a(input_) == 2

[[3 3 3 0 3 3 3]
 [3 3 3 1 3 3 3]
 [3 3 3 2 3 3 3]
 [6 5 4 3 4 5 6]
 [7 3 3 3 3 3 7]
 [8 3 3 3 3 3 8]
 [9 3 3 3 3 3 9]]
[['3' '3' '3' '0' '3' '3' '3']
 ['3' '3' '3' '1' '3' '3' '3']
 ['3' '3' '3' '2' '3' '3' '3']
 ['6' '5' '4' '3' '4' '5' '6']
 ['7' '3' '3' '3' '3' '3' '7']
 ['8' '3' '3' '3' '3' '3' '8']
 ['X' '3' '3' '3' '3' '3' 'X']]


In [78]:
input_ = """..90..9
...1.98
...2..7
6543456
765.987
876....
987....""".replace(".", "3")
assert solution_a(input_) == 4

[[3 3 9 0 3 3 9]
 [3 3 3 1 3 9 8]
 [3 3 3 2 3 3 7]
 [6 5 4 3 4 5 6]
 [7 6 5 3 9 8 7]
 [8 7 6 3 3 3 3]
 [9 8 7 3 3 3 3]]
[['3' '3' '9' '0' '3' '3' 'X']
 ['3' '3' '3' '1' '3' 'X' '8']
 ['3' '3' '3' '2' '3' '3' '7']
 ['6' '5' '4' '3' '4' '5' '6']
 ['7' '6' '5' '3' 'X' '8' '7']
 ['8' '7' '6' '3' '3' '3' '3']
 ['X' '8' '7' '3' '3' '3' '3']]


In [80]:
input_ = """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""
solution_a(input_) == 36

[[8 9 0 1 0 1 2 3]
 [7 8 1 2 1 8 7 4]
 [8 7 4 3 0 9 6 5]
 [9 6 5 4 9 8 7 4]
 [4 5 6 7 8 9 0 3]
 [3 2 0 1 9 0 1 2]
 [0 1 3 2 9 8 0 1]
 [1 0 4 5 6 7 3 2]]
[['8' 'X' '0' '1' '0' '1' '2' '3']
 ['7' '8' '1' '2' '1' '8' '7' '4']
 ['8' '7' '4' '3' '0' 'X' '6' '5']
 ['X' '6' '5' '4' 'X' '8' '7' '4']
 ['4' '5' '6' '7' '8' 'X' '0' '3']
 ['3' '2' '0' '1' 'X' '0' '1' '2']
 ['0' '1' '3' '2' 'X' '8' '0' '1']
 ['1' '0' '4' '5' '6' '7' '3' '2']]


True

In [81]:
answer_a = solution_a(puzzle.input_data)

[[1 4 5 ... 0 3 6]
 [0 3 4 ... 1 4 5]
 [1 2 5 ... 2 3 4]
 ...
 [2 9 6 ... 6 5 0]
 [1 2 5 ... 7 8 9]
 [0 3 4 ... 6 7 8]]
[['1' '4' '5' ... '0' '3' '6']
 ['0' '3' '4' ... '1' '4' '5']
 ['1' '2' '5' ... '2' '3' '4']
 ...
 ['2' 'X' '6' ... '6' '5' '0']
 ['1' '2' '5' ... '7' '8' 'X']
 ['0' '3' '4' ... '6' '7' '8']]


In [128]:
answer_a[2]

6200294120911

In [82]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


## Part Two


In [162]:
small_example = """T.........
...T......
.T........
..........
..........
..........
..........
..........
..........
.........."""

In [166]:
def solution_b(data: str) -> tuple[str, str, int]:
    # This time, attempt to move whole files to the leftmost span of free space blocks
    # that could fit the file. Attempt to move each file exactly once
    # in order of decreasing file ID number starting with the file with the highest file ID number. 
    # If there is no span of free space to the left of a file that is large enough to fit the file,
    # the file does not move.
    file_blocks = list(map(int, data[::2]))
    free_space_blocks = list(map(int, data[1::2]))
    FRAGMENTED_PATTERN = re.compile(r"\d\.+\d")

    block_representation = []

    for i, (file_block, free_space_block) in enumerate(
        zip_longest(file_blocks, free_space_blocks)
    ):
        if file_block:
            block_representation.append((i, file_block))

        if free_space_block:
            block_representation.append((".", free_space_block))

    # block_representation looks like [(0, 4), (., 3), (1, 4), (., 1)] = 4341

    str_block_representation = "".join("".join(str(x[0]) * x[1]) for x in block_representation)

    # continue defragmenting until all gaps are closes
    ordered_block_representation = block_representation.copy()
    
    # go through all file blocks from end to beginning
    right_pos = len(ordered_block_representation) - 1
    while right_pos > 0:
        # find next file block from end
        # at the end first block with pos 0 is always != "."
        while True:
            right = ordered_block_representation[right_pos]
            if right[0] != ".":
                break
            right_pos -= 1
        
        # find next free space block from beginning that can hold file block
        left_pos = 0
        while left_pos < right_pos:
            left = ordered_block_representation[left_pos]
            if left[0] == "." and left[1] >= right[1]:
                break
            left_pos += 1

        # if we cannot find a free space block, move to the next file block
        if left_pos >= right_pos:
            right_pos -= 1
            continue               

        # move whole file block into free space
        space_diff = left[1] - right[1]
        if space_diff == 0:
            ordered_block_representation[left_pos] = (right[0], right[1])
        elif space_diff > 0:
            ordered_block_representation[left_pos] = (right[0], right[1])
            ordered_block_representation.insert(left_pos + 1, (".", space_diff))
            right_pos += 1 # we inserted one element

        # by moving the whole block, we free the space at the original place
        ordered_block_representation[right_pos] = (".", right[1])
        
    str_ordered_block_representation = "".join("".join(str(x[0]) * x[1]) for x in ordered_block_representation)
    
    checksum = 0
    pos = 0
    for block in ordered_block_representation:
        if block[0] == ".":
            pos += block[1]
            continue
        for _ in range(block[1]):
            checksum += block[0] * pos
            pos += 1

    return str_block_representation, str_ordered_block_representation, checksum

In [167]:
solution_b(puzzle.examples[0].input_data)

('00...111...2...333.44.5555.6666.777.888899',
 '00992111777.44.333....5555.6666.....8888..',
 2858)

In [165]:
solution_b(puzzle.examples[0].input_data)[1].strip(".") == "00992111777.44.333....5555.6666.....8888"

00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..


True

In [168]:
assert solution_b(puzzle.examples[0].input_data)[2] == 2858

In [ ]:
answer_b = solution_b(puzzle.input_data)
answer_b

In [170]:
puzzle.answer_b = answer_b[2]

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 9! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
